In [38]:
import pandas as pd
import numpy as np
from myutils import remove_nonnumeric

In [36]:
url = "data/cars-dataset1.csv"
df = pd.read_csv(url)

In [37]:
df.head()

,Car Names,Mileages,Ratings,Reviews,used/certified,Price
0,2019 Honda Accord Sport,Mileage,3.7,(3 reviews),Used,"$27,995.00"
1,2015 Chevrolet Corvette Stingray Z51,"29,192 mi.",3.7,(3 reviews),Used,"$52,000.00"
2,2020 Jeep Grand Cherokee Overland,"53,561 mi.",4.8,"(1,147 reviews)",Used,"$41,998.00"
3,2019 Audi A7 3.0T Prestige,"46,664 mi.",4.9,"(1,831 reviews)",Used,"$62,750.00"
4,2017 Ford Mustang GT Premium,"21,726 mi.",4.0,(409 reviews),Used,"$29,995.00"


In [62]:
# initialize features for filtered dataset
new_df = {}
names = ['name', 'year', 'mileage', 'used/cert', 'rating', 'reviews', 'price']

for name in names:
    new_df[name] = []
    
i=0
# filter data and put into new data file (new_df)
for sample in df.values:
    
    # Ignore rows that have Mileage = 'Mileage' or Price = 'Not Priced'
    if sample[1] == 'Mileage' or sample[5] == 'Not Priced':
        continue
    
    new_df['name'].append(sample[0][5:])
    new_df['year'].append(int(sample[0][0:4]))
    new_df['mileage'].append(int(remove_nonnumeric(sample[1])))
    new_df['rating'].append(float(sample[2]))
    new_df['reviews'].append(int(remove_nonnumeric(sample[3])))
    
    # used -> 0 ; certified -> 1
    if sample[4] == 'Used':
        new_df['used/cert'].append(0)
    else:
        new_df['used/cert'].append(1)

    new_df['price'].append(int(remove_nonnumeric(sample[5][:-3])))

In [73]:
# A look at the filter
df = pd.DataFrame(new_df)
df.to_csv()
df

,name,year,mileage,used/cert,rating,reviews,price
0,Chevrolet Corvette Stingray Z51,2015,29192,0,3.7,3,52000
1,Jeep Grand Cherokee Overland,2020,53561,0,4.8,1147,41998
2,Audi A7 3.0T Prestige,2019,46664,0,4.9,1831,62750
3,Ford Mustang GT Premium,2017,21726,0,4.0,409,29995
4,Jeep Grand Cherokee Limited,2016,66514,0,4.6,1348,26499
...,...,...,...,...,...,...,...
9123,Lexus IS 300 Base,2019,61106,0,4.7,4658,36500
9124,Kia Sorento L,2019,11132,0,4.8,505,23450
9125,GMC Acadia Denali,2017,55074,0,3.4,127,29381
9126,Honda Pilot Elite,2019,57150,1,4.9,8222,38488


In [67]:
# for testing ; check individual row
# for key,val in new_df.items():
#     print(key,val[26],type(val[26]))